In [ ]:
import transformers
import accelerate
import peft
from PIL import ImageFile,Image
from datasets import load_dataset
from transformers import AutoImageProcessor
import torch
from peft import PeftConfig, PeftModel


print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
model_checkpoint = "/DATA/anurag_2101ai04/Assignment/CV/google/vit-base-patch16-224-in21k-lora-indian_food/"

In [ ]:

dataset = load_dataset("imagefolder", data_dir="./indian-foods-80", split="train")

In [ ]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

In [ ]:

image_processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

In [ ]:
# Take an image from the indian-foods-80 dataset
image_path = "./indian-foods-80/test/aloo_matar/aloo_matar_8.jpg"
image = Image.open(image_path)

In [ ]:
# image_processor = AutoImageProcessor.from_pretrained(repo_name)
encoding = image_processor(image.convert("RGB"), return_tensors="pt")

In [ ]:
from transformers import AutoModelForImageClassification

inference_model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

In [ ]:
with torch.no_grad():
    outputs = inference_model(**encoding)
    logits = outputs.logits

predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", inference_model.config.id2label[predicted_class_idx])